# Load llama2-13b-chat for Q&A task (baseline 1)




## 0. Set up the environment

In [ ]:
# # Specify package versions, otherwise install the latest versions
# !pip install accelerate==0.23.0 bitsandbytes==0.41.1 peft==0.4.0 safetensors==0.4.0 transformers==4.34.0 trl==0.4.7 # datasets==2.13.0  # openai==0.28.1


In [ ]:
!pip install -q transformers trl peft  bitsandbytes accelerate  # huggingface_hub datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.1/244.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00


In [ ]:


# load using A100 40G, Gpu usage 26G

import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# define model ID
model_id = "NousResearch/Llama-2-13b-chat-hf"

# load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Mount at gdrive to load data
from google.colab import drive
drive.mount('/content/gdrive')   # connect to gdrive

# Change to current working directory
import os
os.chdir('/content/gdrive/My Drive/Practice/4/ml-arxiv-papers/llama2-13b-chat')  # Make all the following ./path is relative to this path


Mounted at /content/gdrive


In [ ]:
# load the data locally

import json

# load the whole dataset
with open('/content/gdrive/My Drive/Practice/4/ml-arxiv-papers/alldata.json', 'r', encoding='utf-8') as file:  # open from absolute path 绝对路径
    alldata = json.load(file)
print("# Total alldata samples:", len(alldata))
print()

# split test set
test_dataset = alldata[42000:43000]  # alldata[42000:43000]
print("# test_dataset samples:", len(test_dataset))
print("test_dataset 1st sample:", test_dataset[0])




# Total alldata samples: 43713

# test_dataset samples: 1000
test_dataset 1st sample: {'id': 112194, 'context': 'Neural Mesh: Introducing a Notion of Space and Conservation of Energy to   Neural Networks.Neural networks are based on a simplified model of the brain. In this project, we wanted to relax the simplifying assumptions of a traditional neural network by making a model that more closely emulates the low level interactions of neurons. Like in an RNN, our model has a state that persists between time steps, so that the energies of neurons persist. However, unlike an RNN, our state consists of a 2 dimensional matrix, rather than a 1 dimensional vector, thereby introducing a concept of distance to other neurons within the state. In our model, neurons can only fire to adjacent neurons, as in the brain. Like in the brain, we only allow neurons to fire in a time step if they contain enough energy, or excitement. We also enforce a notion of conservation of energy, so that a neuron canno

In [ ]:
sample = test_dataset[0]

# From experience: during inference, exclude <s> and </s> in the prompt, as the model will generate them as special tokens
prompt = f"""[INST] <<SYS>>
You are a helpful, respectful and honest assistant. \
Your task is to generate an answer to the given question. \
And your answer should be based on the provided context only.
<</SYS>>

### Question: {sample['question']}
### Context: {sample['context']} [/INST]
### Answer:
"""


print(prompt)


[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Your task is to generate an answer to the given question. And your answer should be based on the provided context only.
<</SYS>>

### Question: What is the unique feature of the Neural Mesh architecture?
### Context: Neural Mesh: Introducing a Notion of Space and Conservation of Energy to   Neural Networks.Neural networks are based on a simplified model of the brain. In this project, we wanted to relax the simplifying assumptions of a traditional neural network by making a model that more closely emulates the low level interactions of neurons. Like in an RNN, our model has a state that persists between time steps, so that the energies of neurons persist. However, unlike an RNN, our state consists of a 2 dimensional matrix, rather than a 1 dimensional vector, thereby introducing a concept of distance to other neurons within the state. In our model, neurons can only fire to adjacent neurons, as in the brain. Like in the b

In [ ]:
# encode prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# generate output
outputs = model.generate(input_ids=input_ids, max_new_tokens=128, temperature=0.8)  # max_new_tokens=128, larger max_new_tokens longer inference

# decode output
model_output = tokenizer.decode(outputs[0])
generated_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][len(prompt):]

print('\nModel output:\n', model_output)  # decoded_output
print("___" * 20)

print('\nGround truth:\n', sample['answer'])
print('\nGenerated answer:\n', generated_answer)


# Note, inference speed: less than 10s/sample using A100 40G



Model output:
 <s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Your task is to generate an answer to the given question. And your answer should be based on the provided context only.
<</SYS>>

### Question: What is the unique feature of the Neural Mesh architecture?
### Context: Neural Mesh: Introducing a Notion of Space and Conservation of Energy to   Neural Networks.Neural networks are based on a simplified model of the brain. In this project, we wanted to relax the simplifying assumptions of a traditional neural network by making a model that more closely emulates the low level interactions of neurons. Like in an RNN, our model has a state that persists between time steps, so that the energies of neurons persist. However, unlike an RNN, our state consists of a 2 dimensional matrix, rather than a 1 dimensional vector, thereby introducing a concept of distance to other neurons within the state. In our model, neurons can only fire to adjacent neurons, as in the 

Inference on the whole testset and save the results (1000 samples)

In [2]:
# import json

# # Check if the result file already exists, if not, create an empty list
# if not os.path.exists("./test_dataset_inference_results_base.json"):
#     with open("./test_dataset_inference_results_base.json", "w") as f:
#         json.dump([], f)


# for i in range(len(test_dataset)):
#     sample = test_dataset[i]
#     prompt = f"""[INST] <<SYS>>
# You are a helpful, respectful and honest assistant. \
# Your task is to generate an answer to the given question. \
# And your answer should be based on the provided context only.
# <</SYS>>

# ### Question: {sample['question']}
# ### Context: {sample['context']} [/INST]
# ### Answer:
# """

#     # input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
#     input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
#     outputs = model.generate(input_ids=input_ids, max_new_tokens=128, temperature=0.8)
#     # generated_answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
#     generated_answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][len(prompt):]

#     result = {
#         'id': sample['id'],
#         'question': sample['question'],
#         'ground_truth': sample['answer'],
#         'answer': generated_answer
#     }

#     # Read existing results
#     with open("./test_dataset_inference_results_base.json", "r") as f:
#         existing_results = json.load(f)

#     # Append new result to the existing result list
#     existing_results.append(result)

#     # Write results to file
#     with open("./test_dataset_inference_results_base.json", "w") as f:
#         json.dump(existing_results, f)



In [1]:
# # Load and check the results JSON file

# import json

# with open("./test_dataset_inference_results_base.json", "r") as f:
#     test_dataset_inference_results = json.load(f)
# print("# results samples:", len(test_dataset_inference_results))

# # Check results
# test_dataset_inference_results[-3:]
